In [1]:
import os
import sagemaker
from sagemaker.session import s3_input

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
hyperparams={'epochs': 1,
                 'learning-rate': 0.01,
                 'batch-size': 256,
                 'weight-decay': 2e-4,
                 'momentum': 0.9,
                 'optimizer': 'sgd'}

In [8]:
from sagemaker.tensorflow import TensorFlow

output_path = 's3://sagemaker-container-webinar/'
tf_estimator = TensorFlow(entry_point='cifar10-training-script-sagemaker.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.13', 
                          py_version='py3',
                          script_mode=True,
                          output_path=output_path,
                          hyperparameters=hyperparams)

#train_instance_type = 'ml.p3.2xlarge'

In [9]:
train_path = 's3://sagemaker-container-webinar/cifar10-dataset/train'
val_path = 's3://sagemaker-container-webinar/cifar10-dataset/validation'
eval_path = 's3://sagemaker-container-webinar/cifar10-dataset/eval'

tf_estimator.fit({'training': train_path,'validation': val_path,'eval': eval_path})

Creating tmpm7302z90_algo-1-pemlr_1 ... 
Attaching to tmpm7302z90_algo-1-pemlr_12mdone
algo-1-pemlr_1  | 2019-08-05 04:59:04,249 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-pemlr_1  | 2019-08-05 04:59:04,254 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pemlr_1  | 2019-08-05 04:59:04,420 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pemlr_1  | 2019-08-05 04:59:04,434 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pemlr_1  | 2019-08-05 04:59:04,449 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-pemlr_1  | 2019-08-05 04:59:04,461 sagemaker-containers INFO     Invoking user script
algo-1-pemlr_1  | 
algo-1-pemlr_1  | Training Env:
algo-1-pemlr_1  | 
algo-1-pemlr_1  | {
algo-1-pemlr_1  |     "additional_framework_parameters": {},
algo-1-pemlr_1  |     "channel_input_dirs": {
algo-1-pemlr_1  

KeyboardInterrupt: 

In [ ]:
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type='ml.c5.large')

In [ ]:
tf_estimator.model_data

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(20, 100),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'dense-layer':   IntegerParameter(128, 1024),
    'dropout':       ContinuousParameter(0.2, 0.6)
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=2,
                            objective_type=objective_type)